# Concours MTH3302
#### Polytechnique Montréal
### Projet A2024
----
### Objectif
Prédire **la `consommation` en carburant de voitures récentes**.

### Données
Le jeu de données contient pour presque 400 véhicule, la consommation moyenne en L/100km, l'année de frabrication, le type de véhicule, le nombre de cylindre, cylindree, la transmission et la boite.

- `train.csv` est l'ensemble d'entraînement
- `test.csv` est l'ensemble de test


## Chargement des données

Importation des librairies utilisées dans le calepin.

In [ ]:
# import Pkg; Pkg.add("Combinatorics")

In [ ]:
using CSV, DataFrames, Statistics, Dates, Gadfly, Combinatorics, Random, LinearAlgebra, DecisionTree

Premier fichier est l'ensemble des données pour l'entrainement, il contient l'année, le type, le nombre_cylindres, la cylindree, la transmission, la boite, la consommation.

In [ ]:
trainData = CSV.read("./data/train.csv", DataFrame)
trainData.consommation = parse.(Float64,replace.(trainData.consommation, "," => "."))
trainData.cylindree = parse.(Float64,replace.(trainData.cylindree, "," => "."))
trainData[!, :id] = 1:nrow(trainData)
first(trainData, 1)

Le deuxième fichier est l'ensemble des données pour le test, il contient l'année, le type, le nombre_cylindres, la cylindree, la transmission, la boite. Ici il n'a pas la consommation car il s'agit de la variable d'intérêt.

In [ ]:
testData = CSV.read("./data/test.csv", DataFrame)
first(testData, 1)

## Exploration des données

Lors de l’étape d’exploration des données, notre objectif principal était de mieux comprendre les caractéristiques disponibles afin d’évaluer leur pertinence pour la prédiction de la `consommation` en carburant. Nous avons adopté une approche intuitive, en nous appuyant sur nos connaissances de base concernant les véhicules et leur consommation, pour repérer les tendances et des relations potentielles entre les variables.

#### Chargement donnée pour l'exploration

Chargeons les données pour l'exploration

In [ ]:
trainDataExploration = CSV.read("./data/train.csv", DataFrame)
testDataExploration = CSV.read("./data/test.csv", DataFrame)
first(trainDataExploration, 1)

### Données manquantes

Vérifions s'il y a des données manquantes dans l'ensemble d'entrainement et de test.

In [ ]:
function missing_data(data)
    missing_counts = []
    for col_name in names(data)
        missing_count = 0
        for value in data[:, col_name]
            if ismissing(value)
                missing_count += 1
            end
        end
        push!(missing_counts, missing_count) 
    end
    return missing_counts
end

In [ ]:
missing_data(trainDataExploration)

In [ ]:
missing_data(testDataExploration)

Nous pouvons constater qu'il n'y a pas de données manquantes dans les données fournies.

### Conversion des données : préparation des colonnes pour l'analyse et l'évaluation

Étant donné que les colonnes `cylindree` et `consommation` sont actuellement définies comme `String3` et `String31` respectivement, il est nécessaire de modifier leur type en `Float64` dans les deux ensembles de données afin de pouvoir les analyser correctement.

In [ ]:
trainDataExploration.consommation = parse.(Float64,replace.(trainDataExploration.consommation, "," => "."))
trainDataExploration.cylindree = parse.(Float64,replace.(trainDataExploration.cylindree, "," => "."))
first(trainDataExploration, 1)

### Création de nouvelles variables explicatives : 

Pour enrichir notre analyse et améliorer les performances du modèle, nous avons ajouté de nouvelles variables explicatives. 

#### `volume_gaz`
Cette variable est calculée comme le produit du nombre de cylindres et de la cylindrée du moteur. Elle reflète la capacité totale de déplacement du moteur, ce qui pourrait avoir une influence directe sur la consommation de carburant.

In [ ]:
trainDataExploration[!,:volume_gaz] = trainDataExploration[!,:nombre_cylindres] .* trainDataExploration[!,:cylindree]
first(trainDataExploration, 1)

#### `weight`
À compléter
... https://www.insurancenavy.com/average-car-weight/
 https://www.auto-tests.com/fr/lightest-weight/Wagon/all/

In [ ]:
weight_dict = Dict("voiture_moyenne" => 3300, "VUS_petit" => 3500, "voiture_compacte" => 2800, "voiture_deux_places" => 2800, "voiture_minicompacte" => 1500, "VUS_standard" => 5000, "monospace" => 4500, "voiture_sous_compacte" => 2600, "camionnette_petit" => 4200, "break_petit" => 2640, "voiture_grande" => 4400, "camionnette_standard" => 4700, "break_moyen" => 3300)
trainDataExploration[!, :weight] = [weight_dict[t] for t in trainDataExploration[!, :type]]
first(trainDataExploration, 1)


### Recherche de relation entre les variables explicatives et la variable d'intérêt

Analysons maintenant chacune des variables de l'ensemble d'entraînement en fonction de notre variable cible, la `consommation`. Cette étape vise à détecter les relations potentielles entre les variables. Concrètement, nous traçons des graphiques de `consommation` en fonction de différentes variables, telles que : `annee`, `type`, `nombre_cylindres`, `cylindree`, `transmission`, `boite`, `volume_gaz` et `weight`. Nous faisons cela afin de détecter une tendance.

In [ ]:
variables = [:annee, :type, :nombre_cylindres, :cylindree, :transmission, :boite, :volume_gaz, :weight]

plots = [
    Gadfly.plot(
        trainDataExploration,
        x=var,
        y=:consommation,
        Geom.point,
        Guide.xlabel(string(var)),
        Guide.ylabel("consommation")
    ) for var in variables
]

set_default_plot_size(35cm, 35cm)
p = reshape(plots, (4,2))
gridstack(p)


In [ ]:
valeurs_nombre_cylindres = unique(trainDataExploration[:,:nombre_cylindres])
println("Valeurs distinctes de la colonne :nombre_cylindres : ", valeurs_nombre_cylindres)

Analyse des graphiques ci-dessus:

Pour les variables explicatives suivantes en fonction de la consommation:
- `annee` : La distribution de la consommation selon les années semble relativement homogène. On observe cependant une légère diminution de la consommation en 2020 et 2021, probablement attribuable aux effets de la pandémie. Par conséquent, il apparaît que cette variable n'entretient pas de relation linéaire évidente avec la consommation. Cette variable n'aura donc probablement pas d'influence sur la consommation.

- `type`: Pour certains types de véhicules, la quantité de données disponibles est insuffisante, ce qui complique une évaluation fiable et précise de leur impact sur la consommation. Cette limitation justifie la nécessité de mener une analyse complémentaire dédiée à cette variable, comme détaillé dans la prochaine section intitulée "Analyse par type de véhicule". Pour le type voiture_minicompacte, on observe une valeur de consommation nettement plus élevée, ce qui suggère la présence possible de données aberrantes ainsi on va effectuer l'analyse des données aberrantes.

- `nombre_cylindres`: On observe que cette variable peut prendre 7 valeurs distinctes : [3, 4, 5, 6, 8, 10, 12]. Parmi celles-ci, deux valeurs (5 et 12) n'ont chacune qu'une seule donnée de consommation, et une autre valeur (10) en a seulement deux. De plus, on remarque une tendance où l'augmentation du nombre de cylindres est associée à une augmentation de la consommation, suggérant une possible relation linéaire entre ces deux variables. 

- `cylindree` : On remarque une tendance où l'augmentation de la cylindree est associée à une augmentation de la consommation, suggérant une possible relation linéaire entre ces deux variables.

- `transmission`: Les transmissions intégrale, propulsion et 4x4 présentent une distribution de consommation relativement similaire. En revanche, la transmission traction semble afficher une tendance à une consommation inférieure par rapport aux autres.

- `boite`: Le type de boîte de vitesses ne semble pas influencer significativement la consommation, car la distribution de la consommation pour les boîtes automatiques est globalement similaire à celle des boîtes manuelles.

- `volume_gaz` : La relation entre les deux variables semble linéaire. Cependant, on remarque qu'il y a beaucoup plus de données pour des volumes faibles que pour des volumes élevés.

- `weight`: Étant donné que le poids prend en compte les différents types de véhicules, il est difficile d'analyser une relation directe entre le poids et la consommation. Malgré ça, pour le moment on ne voit pas de relation linéaire entre les deux variables.


### Analyse par `type` de véhicule

Nous poursuivons l'exploration en examinant les données par `type`. Cette étape vise à identifier des comportements spécifiques à chaque `type`. En regroupant les véhicules par transmission au sein de chaque `type`, nous calculons la `consommation` moyenne, le `volume_gaz` moyen et le nombre d'observations dans chaque groupe. Cela nous permet de mieux comprendre comment ces facteurs interagissent pour chaque `type` de véhicule et d'évaluer si le `type` ou la `transmission` influence significativement la `consommation`. Cette analyse guide ainsi la sélection des variables pertinentes pour le modèle.

In [ ]:
for type in unique(trainDataExploration.type)
    println(type)
    data_type = trainDataExploration[trainDataExploration.type .== type, :]
    println(combine(groupby(data_type, :transmission), :consommation => mean, :volume_gaz => mean, nrow => :nrow))
    println()
end

TODO:
Ce qu'on en retire de ceci... à compléter 

Nous passons maintenant à une visualisation plus détaillée. Pour chaque `type` de véhicule, nous traçons la relation entre le `volume_gaz` et la `consommation`. Cette étape vise à observer s'il existe des tendances ou des corrélations spécifiques entre ces deux variables pour chaque `type`. Ces visualisations permettent de détecter des comportements distincts ou des patrons dans les données, ce qui peut orienter la sélection des variables explicatives ou révéler des transformations nécessaires pour améliorer la précision du modèle.

In [ ]:
plots = []

for type in unique(trainDataExploration.type)
    data_type = trainDataExploration[trainDataExploration.type .== type, :]
    push!(plots, plot(
        x=data_type.volume_gaz,
        y=data_type.consommation,
        Geom.point,
        Guide.title("Type: $type"),
        Guide.xlabel("Volume Gaz"),
        Guide.ylabel("Consommation")
    ))
end

set_default_plot_size(10cm, 10cm)

for i in 1:length(plots)
    display(plots[i])
end 

Étant donné qu'il y a très peu de données pour certains `type`, il est difficile d'analyser et d'identifier des tendances dans les graphiques ci-dessus.

#### Ajout variable : `general_type`

L'ajout de la variable `general_type` dans les données d'exploration permet de regrouper les types spécifiques de véhicules en catégories plus générales telles que "voiture", "VUS", "camionnette" ou "break". Ces catégories sont basées sur le type de carrosserie de la voiture. Cela simplifie l'analyse en réduisant la granularité de la variable `type`, qui contient de nombreuses sous-catégories. En utilisant ces regroupements, on peut mieux voir les tendances globales et réduire la complexité du modèle de prédiction, tout en conservant les différences majeures entre les groupes de véhicules. 

In [ ]:
general_type_dict = Dict("voiture_moyenne" => "voiture", "VUS_petit" => "VUS", "voiture_compacte" => "voiture", "voiture_deux_places" => "voiture", "voiture_minicompacte" => "voiture", "VUS_standard" => "VUS", "monospace" => "camionnette", "voiture_sous_compacte" => "voiture", "camionnette_petit" => "camionnette", "break_petit" => "break", "voiture_grande" => "voiture", "camionnette_standard" => "camionnette", "break_moyen" => "break")

trainDataExploration[!, :general_type] = [general_type_dict[t] for t in trainDataExploration[!, :type]]
unique(trainDataExploration[!, :general_type])

Examinons maintenant le `volume_gaz` par rapport à la `consommation` pour chacun des `general_type`.

In [ ]:
plots = []

for general_type in unique(trainDataExploration.general_type)
    data_type = trainDataExploration[trainDataExploration.general_type .== general_type, :]
    push!(plots, plot(
        x=data_type.volume_gaz,
        y=data_type.consommation,
        Geom.point,
        Guide.title("General Type: $general_type"),
        Guide.xlabel("Volume Gaz"),
        Guide.ylabel("Consommation")
    ))
end

set_default_plot_size(10cm, 10cm)

for i in 1:length(plots)
    display(plots[i])
end

Les graphiques ci-dessus semble montrer une tendance linéaire, où une augmentation du volume de gaz (x) semble être associée à une hausse de la consommation (y). Ainsi, la variable `general_type` pourrait être une variable explicative pertinente pour comprendre la consommation.

### Transformation des variables explicatives: inclusion des variables qualitatives

L'encodage des variables explicatives qualitatives `general_type`, `transmission` et `boite` est essentiel afin d'être en mesure de les transformer en format numérique et de pouvoir les intégrer dans nos modèles pour effectuer la prédiction.

Nous avons décidé de faire un encodage One-Hot Encoding pour représenter chaque catégorie de manière binaire (0 ou 1), ce qui permet de conserver la distinction entre les différentes catégories tout en évitant de leur attribuer une hiérarchie implicite.

In [ ]:
function encode(data, column)
    for c in unique(data[!, column])
        data[!, Symbol(c)] = ifelse.(data[!, column] .== c, 1, 0)
    end
    return data
end

function encode_data(data)
    encoded_data = deepcopy(data)
    encoded_data = encode(encoded_data, :general_type)
    encoded_data = encode(encoded_data, :transmission)
    encoded_data = encode(encoded_data, :boite)
    return encoded_data
end

function removeRows(data)
    return select!(data, Not([:type, :transmission, :boite, :general_type]))
end

In [ ]:
trainDataExplorationEncoded = encode_data(trainDataExploration)
removeRows(trainDataExplorationEncoded)
first(trainDataExplorationEncoded, 5)

### Analyse de colinéarité

La colinéarité survient lorsque deux ou plusieurs variables sont fortement corrélées, ce qui peut causer des problèmes lors de l'ajustement de modèles statistiques ou d'apprentissage automatique. Plus précisément, une forte colinéarité peut :
- Rendre difficile l'interprétation des coefficients dans un modèle.
- TODO: Diminuer la précision des prédictions en amplifiant les erreurs.
- Affecter la stabilité numérique des algorithmes de régression.

Pour identifier les variables colinéaires, nous calculons une matrice de corrélation pour les variables numériques et repérons les paires dont la corrélation absolue dépasse un certain seuil (ici 0,8). Ceci nous guidera dans le choix des variables explicatives.

In [ ]:
function evaluer_colinearite(data, seuil)
    numerical_data = select(data, findall(x -> eltype(x) <: Number, eachcol(data)))
    
    correlation_matrix = cor(Matrix(numerical_data))
    
    variables_colineaires = []
    for i in 1:size(correlation_matrix, 1)
        for j in (i + 1):size(correlation_matrix, 2)
            if abs(correlation_matrix[i, j]) >= seuil
                push!(variables_colineaires, (names(numerical_data)[i], names(numerical_data)[j]))
            end
        end
    end
    
    return correlation_matrix, variables_colineaires
end

In [ ]:
correlation_matrix, variables_colineaires = evaluer_colinearite(trainDataExplorationEncoded, 0.8)

println("Matrice de corrélation :")
println(correlation_matrix)

println("\nPaires de variables suceptibles d'être colinéaires :")
for paire in variables_colineaires
    println(paire)
end

L'analyse de colinéarité réalisée ci-dessus sur les données a permis de détecter plusieurs paires de variables susceptibles d’être fortement corrélées, notamment `nombre_cylindres`, `cylindree` et `volume_gaz`. Ce qu’on en retire est que ces variables pourraient introduire de la redondance dans les modèles prédictifs, ce qui peut affecter la stabilité et l’interprétabilité des résultats. Étant donné que la variable d'intérêt est la consommation, il serait pertinent de considérer la suppression de certaines de ces variables colinéaires afin de simplifier le modèle et éviter les problèmes liés à la multicolinéarité.

Calcul VIF pour ceux les paires ci-dessus??

### Données uniques

Nous avons remarqué que, parmi les 396 données, de nombreuses possèdent les mêmes caractéristiques, c'est-à-dire que leurs variables explicatives sont identiques, à l’exception de l’année. Comme mentionné dans l’analyse des graphiques de la section "Recherche de relations entre les variables explicatives et la variable d’intérêt", nous n'avons pas observé de relation entre la consommation et l'année. Ainsi, nous sélectionnons toutes les colonnes de l’ensemble d’entraînement, sauf `année` et `consommation`, afin de regrouper les données ayant des caractéristiques similaires. L’objectif est ensuite de calculer la moyenne de la consommation pour chaque groupe ainsi formé. Puis de former un ensemble ayant que les données uniques. 

In [ ]:
function get_unique_data(data)
    data_without_consommation = select(data, Not(:consommation, :annee))


    unique_data = combine(groupby(data, names(data_without_consommation)), :consommation => mean)
    rename!(unique_data, :consommation_mean => :consommation)
    return unique_data
end

In [ ]:
uniqueTrainDataExploration = get_unique_data(trainDataExploration)

On peut obersver que nous avons maintenant 145 données.

Examinons les relations entre la consommation et les variables explicatives pour déterminer si des changements significatifs sont observés par rapport aux graphiques présentés dans la section "Recherche de relations entre les variables explicatives et la variable d’intérêt".

In [ ]:
variables = [:type, :nombre_cylindres, :cylindree, :transmission, :boite, :volume_gaz]

plots = [
    Gadfly.plot(
        uniqueTrainDataExploration,
        x=var,
        y=:consommation,
        Geom.point,
        Guide.xlabel(string(var)),
        Guide.ylabel("consommation")
    ) for var in variables
]

set_default_plot_size(35cm, 35cm)
p = reshape(plots, (3,2))
gridstack(p)

Les graphiques ci-dessus présentent une allure similaire à ceux de la section "Recherche de relations entre les variables explicatives et la variable d’intérêt". Cela dit, il pourrait être intéressant de tester avec les modèles en ne conservant que les données uniques. Cette approche permet de simplifier le dataset en réduisant la redondance des observations ayant les mêmes caractéristiques explicatives. Cela pourrait faciliter l’apprentissage du modèle tout en améliorant son efficacité et en limitant les risques de confusion liés à des consommations différentes pour des données identiques.

### Analyse des données aberrantes

Pour l'analyse des données aberrantes, nous avons commencé par essayer des méthodes classiques avec les quantiles et le score-z. Les résultats étant trop différents d'un type à l'autre, nous avons plutôt opté pour une régression linéaire avec un seuil choisi. Ce choix de régression linéaire a été fait car le graphique de la cylindree en fonction de la consommation suit une tendance linéaire.

On commence par trouver les indices (lignes) des données aberrantes avec la méthode get_outliers_ind_regression_lin :

In [ ]:
function get_outliers_ind_regression_lin(data, x_col, y_col; threshold=2.5)
    slope, intercept = linear_regression(data, x_col, y_col)
    
    x = data[!, x_col]
    y = data[!, y_col]
    
    y_pred = slope .* x .+ intercept
    
    residuals = abs.(y .- y_pred)
    
    residuals_std = std(residuals)
    
    outlier_indices = findall(residuals .> threshold * residuals_std)
    
    outliers = data[outlier_indices, :]
    
    return outlier_indices
end

Ensuite, une méthode pour obtenir ces données aberrantes seulement (get_outliers_regression_lin) :

In [ ]:
function get_outliers_regression_lin(data, outliers_indices)
    outliers = data[outliers_indices, :]
    return outliers
end;

De plus, une méthode pour les séparer des données régulières (remove_outliers_regression_lin):

In [ ]:
function remove_outliers_regression_lin(data::DataFrame, x_col::Symbol, y_col::Symbol; threshold=2.5)
    outlier_indices = get_outliers_ind_regression_lin(data, x_col, y_col, threshold=threshold)

    # println("Outlier Indices Identified: ", outlier_indices)

    keep_mask = trues(nrow(data))
    
    if !isempty(outlier_indices)
        keep_mask[outlier_indices] .= false
    end

    cleaned_data = data[keep_mask, :]
    
    return cleaned_data
end

In [ ]:
function linear_regression(data, x_col, y_col)
    # Extract x and y data from the DataFrame
    x = data[!, x_col]
    y = data[!, y_col]
    
    # Ensure there is enough data for the calculation
    n = length(x)
    if n == 0
        error("Cannot compute linear regression with zero elements.")
    end
    
    # Calculate the mean values
    x_mean = mean(x)
    y_mean = mean(y)
    
    # Calculate slope and intercept for linear regression
    slope = sum((x .- x_mean) .* (y .- y_mean)) / sum((x .- x_mean).^2)
    intercept = y_mean - slope * x_mean
    
    return slope, intercept
end

Dernièrement, une méthode pour visualiser les données avant et après le retrait des données aberrantes (plot_outliers):

In [ ]:

function plot_outliers(uniqueD)
    # Step 2: Find and Display Outliers with Regression
    outliers_indices = get_outliers_ind_regression_lin(uniqueD, :cylindree, :consommation)
    outliers_regression = get_outliers_regression_lin(uniqueD, outliers_indices)
   
    # Step 3: Plot Original Data, Regression Line, and Outliers
    slope, intercept = linear_regression(uniqueD, :cylindree, :consommation)
    regression_line_y = slope .* uniqueD.cylindree .+ intercept
   
    layer_original = layer(x=uniqueD.cylindree, y=uniqueD.consommation, color=uniqueD.type, Theme(default_color="blue"))
    layer_linear_regression = layer(x=uniqueD.cylindree, y=regression_line_y, Geom.line, Theme(default_color="green"))
    layer_regression_outliers = layer(x=outliers_regression.cylindree, y=outliers_regression.consommation, Geom.point, Theme(default_color="red"))
    set_default_plot_size(15cm, 15cm)
    
    display(plot(layer_regression_outliers, layer_original, layer_linear_regression, Guide.xlabel("Cylindree"), Guide.ylabel("Consommation"), Guide.title("Original Data with Outliers")))
   
    # Step 4: Remove Outliers Based on Regression Line
    cleaned_data = remove_outliers_regression_lin(uniqueD, :cylindree, :consommation, threshold=2.5)
   
    # Step 5: Plot Cleaned Data with New Regression Line
    slope_cleaned, intercept_cleaned = linear_regression(cleaned_data, :cylindree, :consommation)
    regression_line_y_cleaned = slope_cleaned .* cleaned_data.cylindree .+ intercept_cleaned
   
   layer_cleaned = layer(x=cleaned_data.cylindree, y=cleaned_data.consommation, color=cleaned_data.type, Theme(default_color="blue"))
   layer_linear_regression_cleaned = layer(x=cleaned_data.cylindree, y=regression_line_y_cleaned, Geom.line, Theme(default_color="green"))
   
   # Display plot for cleaned data
   display(plot(layer_cleaned, layer_linear_regression_cleaned, 
                Guide.xlabel("Cylindree"), Guide.ylabel("Consommation"), Guide.title("Cleaned Data with Regression Line")))
   
   end

In [ ]:
plot_outliers(uniqueTrainDataExploration)

## Choix variables

Suite à l'exploration des données, nous allons construire un DataFrame avec toutes les variables explicatives que nous avons évaluer comme étant pertinente pour la prédiction de la consommation....

### Standardisation des données

Nous avons standardisé les données numériques pour mettre toutes nos données sur la même échelle. De cette façon, on peut comparer leur importance relative. De plus, une variable ayant des valeurs très hautes comparées aux autres n'influencera pas les données de façon plus significative qu'elle le devrait.

In [ ]:
COMSOMMATION_MEAN = mean(trainData.consommation)
COMSOMMATION_STD = std(trainData.consommation)

In [ ]:
function standardize(data)
    return (data .- mean(data)) ./ std(data)
end

function standardize_data(data)
    stddata = deepcopy(data)
   for col in names(stddata)
        if eltype(stddata[!, col]) <: Number && col != "id"
            stddata[!, col] = standardize(stddata[!, col])
        end
    end
    return stddata
end

In [ ]:
function add_rows(data)
    data[!,:volume_gaz] = data[!,:nombre_cylindres] .* data[!,:cylindree]

    #https://www.insurancenavy.com/average-car-weight/
    #https://www.auto-tests.com/fr/lightest-weight/Wagon/all/
    weight_dict = Dict("voiture_moyenne" => 3300, "VUS_petit" => 3500, "voiture_compacte" => 2800, "voiture_deux_places" => 2800, "voiture_minicompacte" => 1500, "VUS_standard" => 5000, "monospace" => 4500, "voiture_sous_compacte" => 2600, "camionnette_petit" => 4200, "break_petit" => 2640, "voiture_grande" => 4400, "camionnette_standard" => 4700, "break_moyen" => 3300)
    data[!, :weight] = [weight_dict[t] for t in data[!, :type]]

    general_type_dict = Dict("voiture_moyenne" => "voiture", "VUS_petit" => "VUS", "voiture_compacte" => "voiture", "voiture_deux_places" => "voiture", "voiture_minicompacte" => "voiture", "VUS_standard" => "VUS", "monospace" => "camionnette", "voiture_sous_compacte" => "voiture", "camionnette_petit" => "camionnette", "break_petit" => "break", "voiture_grande" => "voiture", "camionnette_standard" => "camionnette", "break_moyen" => "break")
    data[!, :general_type] = [general_type_dict[t] for t in data[!, :type]]
    
    return data
end

In [ ]:
function getStandardEncodedData(data)
    data_copy = deepcopy(data)
    standardised_data = get_unique_data(data_copy)
    standardised_data = add_rows(data_copy)
    standardised_data = standardize_data(data_copy)
    standardised_data = encode_data(standardised_data)
    standardised_data = removeRows(standardised_data)
    
    return standardised_data
end


## Évaluation des modèles

### calcul rmse

Nous avons évalué nos modèles avec la métrique rmse, car c'est celle-ci qui est utilisée pour évaluer nos prédictions dans le concours.

In [ ]:
function rmse(y, prediction)
    return sqrt(mean((prediction .- y).^2))
end

Nous avons calculé le rmse moyen en faisant une séparation aléatoire des données à chaque fois afin d'avoir une idée générale du rmse de chaque modèle. 

In [ ]:
function evaluate_rmse(data, model, nrange = 1000, test_size = 0.2, param = 0.0, should_print = true)
    n = 0
    for i in range(0, 1, length=nrange)
        train_data, test_data = train_test_split(data, test_size)
        if (param == 0.0)
            n += model(train_data, test_data)[1]
        else
        n += model(train_data, test_data, param)[1]
        end
    end
    average_rmse = n/nrange
    if should_print
        print("average rmse: ", average_rmse, "\n")
    end
    return average_rmse
end

### Séparation des ensemble d'entrainement et de validation

Nous avons effectué les tests sur nos modèles en prenant soin de séparé les données en ensemble d'entraînement et en ensemble de test. Par défaut, nos données sont séparées 80% dans l'ensemble d'entraînement et 20% dans l'ensemble de test.

In [ ]:
function train_test_split(data, test_size=0.2, shuffle=true)
    n = size(data, 1)
    test_size = floor(Int, n * test_size)
    
    if shuffle
        indices = randperm(n)
    else
        indices = 1:n
    end
    
    test_indices = indices[1:test_size]
    train_indices = indices[test_size+1:end]
    
    train_data = data[train_indices, :]
    test_data = data[test_indices, :]
    
    return train_data, test_data
end

### Les différents modèles

Voici une liste des toutes les modèles prédictifs que nous avons implémenter et tester afin d'obtenir les meilleures prédictions de consommation d'essence:
- Régression linéaire
- Régression ridge
- Régression SVD
- Régression polynomiale
- Arbres de classification
- Forêt de classification

Voici leurs implémentations:

#### Régression linéaire

Le premier modèle que nous avons implémenté est la régression linéaire. Ce modèle nous semblait intuitif, car les données explicatives cylindree, nombre_cylindre et volume_gaz semblent suivre une relation linéaire.

In [ ]:
function regression(training_data, test_data = nothing)	
    X_train =  Matrix(training_data[:, Not(:consommation, :id)])
    y_train = training_data[:, :consommation]

    beta = X_train \ y_train

    rmseval = 0.0
    if test_data != nothing
        X_test = Matrix(test_data[:, Not(:consommation, :id)])
        y_test = test_data[:, :consommation]
        y_predict =  X_test * beta
        y_predict = (y_predict .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        rmseval = rmse(y_test, y_predict)
    end
    
    return rmseval, beta
end

In [ ]:
evaluate_rmse(getStandardEncodedData(trainData), regression)

On peut voir que le rmse moyen obtenu est relativement bas. Le modèle a donc du potentiel et pourrait peut-être être amélioré. Cependant, lorsque nous avons fait une remise officielle sur Kaggle, notre résultat a été nettement supérieur. On peut donc penser que le modèle ne prend pas en compte les données extrêmes et les tendances différentes avec des combinaisons de données spécifiques. On peut donc conclure que la régression linéaire n'est pas la meilleure méthode pour prédire la consommation d'essence.

#### Régression ridge

Le deuxième modèle que nous avons implémenté est la régression ridge. Ce modèle permet de réduire l'importance de la colinéarité entre les données et ainsi réduire la variance du modèle. Cependant, cela introduit un biais dans le modèle. En atteignant un équilibre entre le biais introduit et la diminution de la variance, il est possible de minimiser le rmse. 

In [ ]:
function ridge_regression(training_data, test_data = nothing, lambda=0.5)
    X_train = Matrix(training_data[:, Not([:consommation, :id])])
    y_train = training_data[:, :consommation]
    beta = (X_train'X_train + lambda*I)\X_train'y_train

    rmseval = 0.0
    if test_data != nothing
        X_test = Matrix(test_data[:, Not(:consommation, :id)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        rmseval = rmse(y_test, ychap)
    end
    return rmseval, beta
end

On effectue le calcul du rmse avec plusieurs lambda différent afin de trouver le meilleur lambda possible.

In [ ]:
rmse_data = DataFrame(lambda=Float64[], rmse=Float64[])
for i in 0.1:0.1:40.0
    rmseval = evaluate_rmse(getStandardEncodedData(trainData), ridge_regression, 1000, 0.2, i, false)
    push!(rmse_data, (i, rmseval))
end
display(plot(rmse_data, x=:lambda, y=:rmse, Geom.line, Guide.xlabel("Lambda"), Guide.ylabel("RMSE")))

filtered_rmse_data = filter(row -> row[:lambda] < 10, rmse_data)
display(plot(filtered_rmse_data, x=:lambda, y=:rmse, Geom.line, Guide.xlabel("Lambda"), Guide.ylabel("RMSE")))

best_row = rmse_data[argmin(rmse_data.rmse),:]
println("meilleur lambda: ", best_row.lambda)
println("rmse correspondant: ", best_row.rmse)

On peut voir qu'il y a beaucoup de variation dans la valeur de rmse dépendamment de la valeur de lambda. Cette variation est causé en partie par la manière aléatoire avec laquelle les données sont séparées en ensemble d'entraînement et de test. On voit toutefois une tendance à la hausse lorsque lambda dépasse 10. On a donc fait un deuxième graphique pour mieux voir les données entre 0.1 et 10. On a ensuite trouvé la valeur de lambda qui donne le meilleur rmse. Ce rmse est comparable à celui de la régression linéaire. La régression ridge souffre des mêmes problèmes que la régression linéaire. On peut donc conclure que la régression ridge n'est pas la meilleure méthode pour prédire la consommation d'essence.

#### Régression SVD

Le troisième modèle que nous avons implémenté est la régression SVD. Ce modèle permet de décerner s'il pourrait y avoir des variables et des relations entre les variables latentes qui peuvent améliorer nos prédictions.

In [ ]:
function svd_regression(training_data, test_data = nothing)
    X_train = Matrix(training_data[:, Not([:consommation, :id])])
    y_train = training_data[:, :consommation]
    
    U, S, V = svd(X_train)

    beta = V' * Diagonal([s > 1e-10 ? 1/s : 0 for s in S]) * U' * y_train

    rmseval = 0.0
    if test_data != nothing
        X_test = Matrix(test_data[:, Not(:consommation,:id)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        rmseval = rmse(y_test, ychap)
    end
    return rmseval, beta
end

In [ ]:
evaluate_rmse(getStandardEncodedData(trainData), svd_regression)

On remarque que le modèle de régression SVD obtient un rmse très élevé. Avec les données que nous avons, il semble que le modèle de régression SVD surcomplexifie le modèle et donc nuit aux résulats. On peut donc conclure que la régression SVD n'est pas la meilleure méthode pour prédire la consommation d'essence.

#### régression polynomiale

Le quatrième modèle que nous avons implémenté est la régression polynomiale. Ce modèle permet de modéliser des relations non-linéaires. 

In [ ]:
function construct_structure(x::Matrix{<:Real}, order::Int)
    n, m = size(x)
    poly_terms = [x[:, j].^p for j in 1:m, p in 0:order]
    X = hcat(poly_terms...)
    return X
end

function polynomial_regression(training_data, test_data = nothing, degree = 3)
    X_train = construct_structure(Matrix(training_data[:, Not([:consommation, :id])]), degree)
    y_train = training_data[:, :consommation]

    beta = X_train \ y_train

    rmseval = 0.0

    if test_data != nothing
        X_test = construct_structure(Matrix(test_data[:, Not([:consommation, :id])]), degree)
        y_test = test_data[:, :consommation]
        ychap = X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN

        rmseval = rmse(y_test, ychap)
    end

    return rmseval, beta
end

In [ ]:
evaluate_rmse(getStandardEncodedData(trainData), polynomial_regression)

On peut voir que le rmse moyen obtenu est relativement bas. Le modèle a donc du potentiel et pourrait peut-être être amélioré. Cependant, lorsque nous avons fait une remise officielle sur Kaggle, notre résultat a été nettement supérieur. On peut donc penser que le modèle ne prend pas en compte les données extrêmes et les tendances différentes avec des combinaisons de données spécifiques. On peut donc conclure que la régression polynomiale n'est pas la meilleure méthode pour prédire la consommation d'essence.

#### Arbre de régression

Le cinquième modèle que nous avons implémenté est un arbre de régression. Ce modèle permet d'effectuer des séparations dans l'ensemble d'entraînement afin de discerner des liens entre certaines variables et d'ainsi mieux prédire les cas extrêmes et les combinaisons de données spécifiques. Ce modèle tente donc de pallier aux problèmes des modèles linéaire, ridge et polynomiale.

In [68]:
@doc DecisionTreeRegressor

```
DecisionTreeRegressor(; pruning_purity_threshold=0.0,
                      max_depth::Int-1,
                      min_samples_leaf::Int=5,
                      min_samples_split::Int=2,
                      min_purity_increase::Float=0.0,
                      n_subfeatures::Int=0,
                      rng=Random.GLOBAL_RNG,
                      impurity_importance::Bool=true)
```

Decision tree regression. See [DecisionTree.jl's documentation](https://github.com/bensadeghi/DecisionTree.jl)

Hyperparameters:

  * `pruning_purity_threshold`: (post-pruning) merge leaves having `>=thresh` combined purity (default: no pruning). This accuracy-based method may not be appropriate for regression tree.
  * `max_depth`: maximum depth of the decision tree (default: no maximum)
  * `min_samples_leaf`: the minimum number of samples each leaf needs to have (default: 5)
  * `min_samples_split`: the minimum number of samples in needed for a split (default: 2)
  * `min_purity_increase`: minimum purity needed for a split (default: 0.0)
  * `n_subfeatures`: number of features to select at random (default: keep all)
  * `rng`: the random number generator to use. Can be an `Int`, which will be used to seed and create a new random number generator.
  * `impurity_importance`: whether to calculate feature importances using `Mean Decrease in Impurity (MDI)`. See [`DecisionTree.impurity_importance`](@ref)

Implements `fit!`, `predict`, `get_classes`


In [69]:
function tree_regression(training_data, test_data = nothing)
    X_train = Matrix(training_data[:, Not(:consommation, :id)])
    y_train = training_data.consommation
    X_test = Matrix(test_data[:, Not(:consommation,:id)])
    y_test = test_data.consommation

    model = DecisionTreeRegressor(n_subfeatures=12,min_samples_leaf=1,min_purity_increase=0.0, max_depth=5, min_samples_split=6)
    fit!(model, X_train, y_train)
    ychap =  predict(model, X_test)  

    ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
    y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
    rmseval = rmse(y_test, ychap)

end

tree_regression (generic function with 2 methods)

In [71]:
evaluate_rmse(getStandardEncodedData(trainData), tree_regression)


average rmse: 0.9434730569908875


0.9434730569908875

On remarque que le résultat est similaire à ceux des modèles linéaire, ridge et polynomiale. Cela peut être expliqué par le fait qu'un arbre de régression comporte généralement une assez grande variance. De plus, si certaines données n'ont pas de données similaires dans l'ensemble d'entraînement, celles-ci peuvent être très difficiles à prédire. Ce modèle pourrait donc être amélioré.

#### Forêt aléatoire de régression

Le sixième modèle que nous avons implémenté est une forêt aléatoire de régression. Ce modèle utilise plusieurs arbre de régression afin de pallier aux problèmes engendrés par le fait d'utiliser un seul arbre de régression. Ce modèle devrait donc amélioré les résultats du modèle précédent.

In [73]:
@doc RandomForestRegressor

```
RandomForestRegressor(; n_subfeatures::Int=-1,
                      n_trees::Int=10,
                      partial_sampling::Float=0.7,
                      max_depth::Int=-1,
                      min_samples_leaf::Int=5,
                      rng=Random.GLOBAL_RNG,
                      impurity_importance::Bool=true)
```

Random forest regression. See [DecisionTree.jl's documentation](https://github.com/bensadeghi/DecisionTree.jl)

Hyperparameters:

  * `n_subfeatures`: number of features to consider at random per split (default: -1, sqrt(# features))
  * `n_trees`: number of trees to train (default: 10)
  * `partial_sampling`: fraction of samples to train each tree on (default: 0.7)
  * `max_depth`: maximum depth of the decision trees (default: no maximum)
  * `min_samples_leaf`: the minimum number of samples each leaf needs to have (default: 5)
  * `min_samples_split`: the minimum number of samples in needed for a split
  * `min_purity_increase`: minimum purity needed for a split
  * `rng`: the random number generator to use. Can be an `Int`, which will be used to seed and create a new random number generator. Multi-threaded forests must be seeded with an `Int`
  * `impurity_importance`: whether to calculate feature importances using `Mean Decrease in Impurity (MDI)`. See [`DecisionTree.impurity_importance`](@ref).

Implements `fit!`, `predict`, `get_classes`


In [ ]:
function random_forest(training_data, test_data = nothing)
    X_train =  Matrix(training_data[:, Not(:consommation, :id)])
    y_train = training_data.consommation
    X_test = Matrix(test_data[:, Not(:consommation, :id)])
    y_test = test_data.consommation


    model = RandomForestRegressor(n_subfeatures=12, n_trees=1000,min_samples_leaf=1,min_purity_increase=0.0, max_depth=5, min_samples_split=6)
    DecisionTree.fit!(model, X_train, y_train)
    ychap = DecisionTree.predict(model, X_test)
    y_test = (y_test * COMSOMMATION_STD) .+ COMSOMMATION_MEAN
    ychap = (ychap * COMSOMMATION_STD) .+ COMSOMMATION_MEAN
    return rmse(y_test, ychap)
end

random_forest (generic function with 2 methods)

In [72]:
evaluate_rmse(getStandardEncodedData(trainData), random_forest, 100)

average rmse: 0.8699680936094508


0.8699680936094508

On voit donc que le rmse moyen obtenu est inférieur à celui de tous les modèles précédents. Ce modèle a donc du potentiel et pourrait être amélioré.Cependant, lorsque nous avons fait une remise officielle sur Kaggle, notre résultat a été nettement supérieur. Il y a donc potentiellement des combinaisons de données qui ne se retrouve pas dans les données d'entraînement. C'est une des faiblesse du modèle de forêt aléatoire de régression. On peut donc conclure que la forêt aléatoire de régression n'est pas la meilleure méthode pour prédire la consommation d'essence.

#### Comparaison des modèles

Tous les modèles, à l'exception de la régression SVD qui est nettement pire, ont un rmse moyen relativement semblable et comportent tous des défauts. Bien que l'écart soit petit, le modèle forêt aléatoire de régression a obtenu le meilleur résultat. 

Chacun de ces modèles n'a pas performé à la hauteur des résulats attendus lors de nos remises sur Kaggle. Il faut donc apporter des modifications afin de les rendre meilleurs pour prédire des données qui ne sont pas dans l'ensemble d'entraînement.

Nous avons décider d'implémenter un modèle basé sur les arbres de régression afin de faire notre remise finale. 

## Conclusion et amélioration